In [48]:
#======================IMPORTS==============================================================================================
import pandas as pd
from pandas.io.sql import to_sql
from sqlalchemy import create_engine, types
import json
import requests
from itertools import chain
import mysql.connector



#=============================REQUISICAO=====================================================================================
# URL para fazer a requisição da API.
url = "https://covid-api.mmediagroup.fr/v1/cases"
# Faz a chamada da API com o método request da bilbioteca requests.
# OBS.: Estamos passando o método GET na chamada da API, isso indica que estamos solicitando somente um retorno de informação.
retorno = requests.request("GET", url)


#============================TRANSFORMACAO==================================================================================
# Do retorno que foi recebido e jogado em memória dento da variável retorno queremos somente o valor em texto, por isso passamos o parâmetro ".text".
dados = retorno.text
# Carregamos e transformamos o texto para o formato Json com a função loads() da biblioteca json.
dados = json.loads(dados)
# descompactando os valores do dict 'dados.values()' com a função chain do módulo itertools
df = pd.DataFrame(chain.from_iterable(map(lambda sec: sec.values(), dados.values())))
# remover as colunas que não usaremos
df = df.drop(columns=['sq_km_area', 'life_expectancy', 'elevation_in_meters','continent', 'abbreviation', 'iso', 'capital_city'])
#remover as linhas Nan do campo country
df = df.dropna(subset=['country'])
# formatar o campos updated para o YYYY-MM-DD HH:MM:SS
df['updated'] = pd.to_datetime(df['updated'])
# converter coluna "population" to int64 dtype
df = df.astype({"population": int})

#==========================CONEXAO==========================================================================================
# iniciando a conexão com o banco usando sqlalchemy 
# atenção à string passada na função abaixo:  user  pass        schema
conn = create_engine('mysql+mysqlconnector://root:''@localhost/projeto_api', connect_args={'auth_plugin': 'mysql_native_password'})
# Método to_sql transforma o DataFrame em um insert automaticamente, passando a engine criada acima para conectar ao banco
to_sql(df, 'api_covid', conn, schema='projeto_api', if_exists='append')

#=========================VISUALIZACAO=====================================================================================
# Executando linguagem SQL em Python
total_world_deaths = pd.read_sql('select sum(deaths) from api_covid', conn)
total_location_deaths = pd.read_sql('select sum(deaths), location from api_covid group by location;', conn)
df_covid = pd.read_sql('select country, deaths, deaths / population, recovered / confirmed from api_covid;', conn)

print('Total de mortes no mundo: ', total_world_deaths)
print('Total de mortes por região:', total_location_deaths)

Total de mortes no mundo:     sum(deaths)
0   54765927.0
Total de mortes por região:     sum(deaths)                   location
0     6723073.0  Southern and Central Asia
1     4207568.0            Southern Europe
2      718632.0            Northern Africa
3       40073.0             Central Africa
4      102024.0                  Caribbean
5    11957605.0              South America
6     1866131.0                Middle East
7       14976.0  Australia and New Zealand
8     4166587.0             Western Europe
9     5887373.0             Eastern Europe
10    3969753.0            Central America
11     102266.0             Western Africa
12     920899.0            Southern Africa
13    1151126.0             Southeast Asia
14     235081.0             Eastern Africa
15    9827763.0              North America
16     306523.0               Eastern Asia
17     297803.0           Nordic Countries
18     123542.0           Baltic Countries
19       2544.0                  Melanesia
20    212658

In [36]:
# DATA FRAME APÓS A ETAPA DE TRANSFORMAÇÃO
df

,confirmed,recovered,deaths,country,population,location,lat,long,updated
0,65080,55886,2782,Afghanistan,35530081,Southern and Central Asia,33.93911,67.709953,2021-05-22 10:20:48+00:00
1,132153,128425,2441,Albania,2930187,Southern Europe,41.1533,20.1683,2021-05-22 10:20:48+00:00
2,126434,88066,3405,Algeria,41318142,Northern Africa,28.0339,1.6596,2021-05-22 10:20:48+00:00
3,13569,13234,127,Andorra,76965,Southern Europe,42.5063,1.5218,2021-05-22 10:20:48+00:00
4,31909,26513,709,Angola,29784193,Central Africa,-11.2027,17.8739,2021-05-22 10:20:48+00:00
...,...,...,...,...,...,...,...,...,...
721,5013,2689,41,Vietnam,95540800,Southeast Asia,14.058324,108.277199,2021-05-22 10:20:48+00:00
723,6632,3179,1302,Yemen,28250420,Middle East,15.552727,48.516388,2021-05-22 10:20:48+00:00
724,92920,91019,1266,Zambia,17094130,Eastern Africa,-13.133897,27.849332,2021-05-22 10:20:48+00:00
725,38664,36435,1586,Zimbabwe,16529904,Eastern Africa,-19.015438,29.154857,2021-05-22 10:20:48+00:00


In [46]:
# DATA FRAME IMPORTADO COM AS MÉTRICAS SQL
df_covid

,country,deaths,deaths / population,recovered / confirmed
0,Afghanistan,2772,0.0001,0.8624
1,Albania,2438,0.0008,0.9632
2,Algeria,3395,0.0001,0.6970
3,Andorra,127,0.0017,0.9753
4,Angola,696,0.0000,0.8416
...,...,...,...,...
2665,Vietnam,41,0.0000,0.5364
2666,Yemen,1302,0.0000,0.4793
2667,Zambia,1266,0.0001,0.9795
2668,Zimbabwe,1586,0.0001,0.9423
